In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
def get_article_content(article_url):
    response = requests.get(article_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        article_content_div = soup.find('div', {'id': 'maincontent'})
        if article_content_div:
          excluded_elements = article_content_div.find_all('figure', {'data-spacefinder-role': 'inline', 'data-spacefinder-type': 'model.dotcomrendering.pageElements.NewsletterSignupBlockElement', 'class': 'dcr-173mewl'})
          for elem in excluded_elements:
              elem.decompose()

          if article_content_div:
              paragraphs = article_content_div.find_all('p')
              content = ' '.join(' '.join(p.stripped_strings) if not p.find('span', class_='dcr-1ipjagz') else ''.join(p.stripped_strings) for p in paragraphs)
              content = re.sub(r'\s+', ' ', content).strip()
              return content
    return None

In [ ]:
def get_article_summary(article_url):
    response = requests.get(article_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        summary_element = soup.find('div', {'data-gu-name': 'standfirst'})
        if summary_element:
            for a in summary_element.find_all('a'):
                a.decompose()
            return summary_element.get_text(strip=True)
    return None

In [ ]:
base_url = "https://www.theguardian.com/business/banking?page="
current_page = 1
dataset = []

In [ ]:
while True:
    url = base_url + str(current_page)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        article_links = soup.find_all('a', class_='u-faux-block-link__overlay js-headline-text', href=lambda href: href and 'live' not in href and 'commentisfree' not in href and 'grogonomics' not in href)

        if not article_links:
            break

        for link in article_links:
            article_url = link.get('href')
            headline = link.get_text(strip=True)
            summary = get_article_summary(article_url)
            content = get_article_content(article_url)

            if summary is not None and content is not None:
                dataset.append({
                    'URL': article_url,
                    'Headline': headline,
                    'Summary': summary,
                    'Article': content
                })
            else:
                print(f"Skipping: {article_url}. Summary or article content not found.")

        current_page += 1
        print(current_page)

    else:
        print(f"Failed to retrieve page {current_page}. Status code:", response.status_code)
        break

In [ ]:
df = pd.DataFrame(dataset)
df.to_csv('/content/drive/My Drive/theguardian_economics.csv', index=False, encoding='utf-8')
print('Data saved.')